# Enmascaramiento Nube Landsat 8 TOA google Earth Engine

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Función mascara Landsat 8 TOA

In [2]:
# Funcion mascara Landsat 8 TOA
def cloudMask(image):
  clouds = ee.Algorithms.Landsat.simpleCloudScore(image).select("cloud")
  return (image.updateMask(clouds.lt(35)))

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

## Importar Zona de estudio

In [4]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

## Colleccion Landsat 8 TOA

In [5]:
# Coleccion de Landsat 8 OLI TIRS TOA
L8_toa = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
            .filterDate('2019-01-01','2019-12-31')\
            .filterMetadata('CLOUD_COVER', 'less_than', 20)\
            .filterBounds(zona_ee)


In [6]:
# Conteo de imagenes
count = L8_toa.size().getInfo()
print("Cantidad de imagenes L8_toa:", count)

Cantidad de imagenes L8_toa: 4


## Determinar ID imagen

In [9]:
# Imprimir la lista de ID Imagenes
catalogo_L8 = L8_toa.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
catalogo_L8

['LC08_003069_20190621',
 'LC08_003069_20190808',
 'LC08_003069_20190824',
 'LC08_003069_20190925']

In [10]:
type(catalogo_L8)

list

In [11]:
catalogo_L8[0]

'LC08_003069_20190621'

## Seleccion imagen Landsat 8

In [12]:
# Importar imagen Landsat 8 RS 
image1 = ee.Image('LANDSAT/LC08/C01/T1_TOA' + "/" + catalogo_L8[0])\
            .reproject(crs="EPSG:32719", scale = 30)
image2 = ee.Image('LANDSAT/LC08/C01/T1_TOA' + "/" + catalogo_L8[1])\
            .reproject(crs="EPSG:32719", scale = 30)
image3 = ee.Image('LANDSAT/LC08/C01/T1_TOA' + "/" + catalogo_L8[2])\
            .reproject(crs="EPSG:32719", scale = 30)
image4 = ee.Image('LANDSAT/LC08/C01/T1_TOA' + "/" + catalogo_L8[3])\
            .reproject(crs="EPSG:32719", scale = 30)

In [13]:
# Nombre de las bandas
print(image1.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA']


In [14]:
# Consultar Proyeccion
print("Proyeccion:",image1.projection().getInfo())

Proyeccion: {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}


In [15]:
# Simbologia escala 0 - 1 TOA
viz_toa = {
    'bands': ['B6', 'B5', 'B4'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.5
}

In [16]:
geometria = image1.geometry()
Map.centerObject(geometria)

In [17]:
# Visualizar mapa
Map.addLayer(image1, viz_toa, "L8 img1")
Map.addLayer(image2, viz_toa, "L8 img2")
Map.addLayer(image3, viz_toa, "L8 img3")
Map.addLayer(image4, viz_toa, "L8 img4")
Map

Map(center=[-13.014534688269258, -70.47460127201552], controls=(WidgetControl(options=['position', 'transparen…

## Determinar el enmascaramiento nubosidad

In [18]:
# Realizacion la funcion de nubosidad
img_cloud1 = cloudMask(image1)
img_cloud2 = cloudMask(image2)
img_cloud3 = cloudMask(image3)
img_cloud4 = cloudMask(image4)

In [19]:
# Visualizar mapa
Map.addLayer(img_cloud1, viz_toa, "img_cloud1")
Map.addLayer(img_cloud2, viz_toa, "img_cloud2")
Map.addLayer(img_cloud3, viz_toa, "img_cloud3")
Map.addLayer(img_cloud4, viz_toa, "img_cloud4")
Map

Map(bottom=35487.0, center=[-13.170422607388247, -69.9216879175015], controls=(WidgetControl(options=['positio…

## Crear una imagenCollection

In [20]:
# seleccionar varias imagenes
selec_imagenes = ee.ImageCollection.fromImages([img_cloud3,img_cloud2,img_cloud1])

## Crear mosaico maximo

In [21]:
# crear mosaico
mosaico = selec_imagenes.max()

In [22]:
# Visualizar mapa
Map.addLayer(mosaico, viz_toa, "Mosaico")
Map

Map(bottom=35539.0, center=[-13.448394732660567, -69.70275878906251], controls=(WidgetControl(options=['positi…